In [1]:

!pip install web3 requests tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.5/587.5 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 81.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from web3 import Web3
import requests
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler


In [3]:
wallets_df = pd.read_csv("/content/Wallet id - Sheet1.csv")  # Adjust path
print(wallets_df.columns)


Index(['wallet_id'], dtype='object')


In [4]:
wallets = wallets_df['wallet_id'].tolist()

In [5]:
API_KEY = "cqt_rQgrjcjb48y6TQMTW7gcFtpkFhJp"
BASE_URL = "https://api.covalenthq.com/v1/1/address"

def get_wallet_data(wallet):
    url = f"{BASE_URL}/{wallet}/balances_v2/?key={API_KEY}"
    resp = requests.get(url)
    if resp.status_code != 200:
        return {}
    return resp.json()


In [6]:
def extract_features(wallet):
    # For demo, placeholder features:
    return {
        "wallet_id": wallet,
        "wallet_age_days": np.random.randint(10, 1500),
        "tx_count": np.random.randint(1, 300),
        "tx_count_30d": np.random.randint(0, 50),
        "average_gas_used": np.random.randint(10000, 300000),
        "borrow_events": np.random.randint(0, 20),
        "repay_events": np.random.randint(0, 20),
        "liquidation_events": np.random.randint(0, 5)
    }


In [7]:
features = []

for wallet in tqdm(wallets):
    features.append(extract_features(wallet))

df = pd.DataFrame(features)


100%|██████████| 103/103 [00:00<00:00, 8110.95it/s]


In [8]:
from sklearn.preprocessing import MinMaxScaler

# Avoid log(0)
for col in ["wallet_age_days", "tx_count", "tx_count_30d", "average_gas_used"]:
    df[col] += 1

# Log transformation (less skew)
df["log_age"] = np.log1p(df["wallet_age_days"])
df["log_tx"] = np.log1p(df["tx_count"])
df["log_tx_30d"] = np.log1p(df["tx_count_30d"])
df["log_gas"] = np.log1p(df["average_gas_used"])

# Scale features
features_to_scale = ["log_age", "log_tx", "log_tx_30d", "log_gas"]
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df[features_to_scale]), columns=features_to_scale)

# Weighting risk indicators
weights = {
    "log_age": 0.25,
    "log_tx": 0.25,
    "log_tx_30d": 0.2,
    "log_gas": 0.2,
    # optional risk reduction for liquidations
    "liquidation_penalty": 0.1
}

# Apply scoring
df["score"] = (
    1000 * (
        df_scaled["log_age"] * weights["log_age"]
        + df_scaled["log_tx"] * weights["log_tx"]
        + df_scaled["log_tx_30d"] * weights["log_tx_30d"]
        + df_scaled["log_gas"] * weights["log_gas"]
    )
    - df["liquidation_events"] * 20  # penalty per liquidation
)

# Clip minimum and maximum
df["score"] = df["score"].clip(lower=0, upper=1000).astype(int)


In [9]:
df[["wallet_id", "score"]].to_csv("wallet_risk_scores_1.csv", index=False)
